<a href="https://colab.research.google.com/github/Real1CM/MLassignment/blob/main/%E4%BD%9C%E4%B8%9A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
import zipfile
import tarfile

# 解压 .tar.gz 文件
def extract_tar_gz(file_path, extract_path):
    with tarfile.open(file_path, "r:gz") as tar:
        tar.extractall(path=extract_path)

# 解压 .zip 文件
def extract_zip(file_path, extract_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# 文件路径
tar_gz_file = './aclImdb_v1.tar.gz'
train_zip = './train.zip'
test_zip = './test.zip'

# 解压缩路径
base_dir = './aclImdb'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# 解压缩
extract_tar_gz(tar_gz_file, './')
extract_zip(train_zip, './')
extract_zip(test_zip, './')

print("解压完成！")

解压完成！


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 读取文件并提取文本数据
def load_data_from_folder(folder_path, label):
    texts = []
    labels = []
    for file_name in os.listdir(folder_path):
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as f:
            texts.append(f.read())
            labels.append(label)
    return texts, labels

# 加载训练数据
neg_train_folder = os.path.join(train_dir, 'neg')
pos_train_folder = os.path.join(train_dir, 'pos')

neg_train_texts, neg_train_labels = load_data_from_folder(neg_train_folder, 0)
pos_train_texts, pos_train_labels = load_data_from_folder(pos_train_folder, 1)

# 合并数据
train_texts = neg_train_texts + pos_train_texts
train_labels = neg_train_labels + pos_train_labels

# 使用 Keras Tokenizer 来进行分词和创建字典
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

# 将文本转换为基于字典的索引序列
train_sequences = tokenizer.texts_to_sequences(train_texts)

# 统计词汇表大小
vocab_size = len(tokenizer.word_index) + 1
print(f"词汇表大小: {vocab_size}")

# 填充序列，使得每个序列长度相同
max_length = 500  # 设置最大序列长度
X_train = pad_sequences(train_sequences, maxlen=max_length)
y_train = np.array(train_labels)

print(f"训练集样本数量: {X_train.shape[0]}")

词汇表大小: 88583
训练集样本数量: 25000


In [ ]:
# 填充到最大长度
X_train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

# 打印填充后的样本
print(X_train_padded[0])  # 打印第一个样本的填充后序列

[   22    29   121     1    62     4  3417    10    78     7     7    70
     1     5    27    39    21     5    27  5229    21     1  2485   407
    44    74  3584    80     1  1559    35   108   208    12    42    21
    52   218     8   189     9   283    12    84     5   891    16     8
   189    10   166  3417     3    49    18  6108  3866   294    42    21
    57  5674   115 18510 10307     2  6538  6259    23   744   208   125
    16     3 30568 19499   921     2  4988    21    96   227   493 10086
  3285   126  2696   122    24 27631 40830    77    94 17633 21905   215
    12    75   342    47    23   772     4   405    12    23   125    71
    11    31  2278     7     7    10   525    63   926    96    73    20
     1    17   244    10    77 18511     1  1250   524  3169   387     7
     7   860  1946     2     1 13420  1171     3  3146   497    76     5
  1257     1    17    33   939    15  3417  4632  3286    95    11 11037
 11037 11037     2  5399    15     1    88   354   

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# 构建模型
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 打印模型摘要
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 训练模型
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 477s 2s/step - accuracy: 0.6206 - loss: 0.6652 - val_accuracy: 0.0000e+00 - val_loss: 0.9682
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - accuracy: 0.6280 - loss: 0.6591 - val_accuracy: 0.0154 - val_loss: 0.9144
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 479s 2s/step - accuracy: 0.6453 - loss: 0.6427 - val_accuracy: 0.0526 - val_loss: 0.9483
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - accuracy: 0.6547 - loss: 0.6202 - val_accuracy: 0.0442 - val_loss: 1.1164
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 472s 2s/step - accuracy: 0.6544 - loss: 0.6140 - val_accuracy: 0.0456 - val_loss: 0.9596


In [14]:
# 加载测试数据
neg_test_folder = os.path.join(test_dir, 'neg')
pos_test_folder = os.path.join(test_dir, 'pos')

neg_test_texts, neg_test_labels = load_data_from_folder(neg_test_folder, 0)
pos_test_texts, pos_test_labels = load_data_from_folder(pos_test_folder, 1)

# 合并测试数据
test_texts = neg_test_texts + pos_test_texts
test_labels = neg_test_labels + pos_test_labels

# 将测试数据转换为索引序列
test_sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(test_sequences, maxlen=max_length)
y_test = np.array(test_labels)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f"测试集准确率: {accuracy*100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 217s 278ms/step - accuracy: 0.5495 - loss: 1.2798
测试集准确率: 53.01%
